cf : https://www.kaggle.com/alexisemanuelli/fine-tune

In [9]:
!pip install keras==2.15.0 plotly optuna
#Import Os and Basis Libraries
import cv2
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
#Matplot Images
import matplotlib.image as mpimg
# Tensflor and Keras Layer and Model and Optimize and Loss
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import *
from tensorflow.keras.losses import BinaryCrossentropy
# import tensorflow_hub as hub
import optuna
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.applications import ResNet50, InceptionV3, Xception, VGG16
from tensorflow.keras.layers import Dense
import numpy as np
#Image Generator DataAugmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
#Early Stopping
from tensorflow.keras.callbacks import EarlyStopping
# Warnings Remove 
import warnings 
warnings.filterwarnings("ignore")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 9.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 62.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 614.3/614.3 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.4 MB/s eta 0:00:00


/export/home1/users/zfne/emanuell/Documents/GitHub/Dolphins/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Directory containing the "Train" folder
directory = "/media/DOLPHIN_ALEXIS/Analyses_alexis/Spectrograms_datasets/dataset/_last/tests"

# List of categories (subfolder names)
categories = ["negatives", "positives"]

# Initialize lists to store filenames and categories
filenames = []
category_labels = []

# Iterate through the categories
for category in categories:
    # Path to the current category folder
    category_folder = os.path.join(directory, "train", category)
    # List all filenames in the category folder
    category_filenames = os.listdir(category_folder)
    # Append filenames and corresponding category labels
    filenames.extend(category_filenames)
    category_labels.extend([category] * len(category_filenames))

# Create DataFrame
df = pd.DataFrame({
    'filename': filenames,
    'category': category_labels
})

# Display the first few rows of the DataFrame
print(df.head())

                          filename   category
0  Exp_12_Jan_2020_1545pm-26.8.jpg  negatives
1  Exp_23_Nov_2019_1145am-33.6.jpg  negatives
2                2452.8-2453.2.jpg  negatives
3                1594.0-1594.4.jpg  negatives
4  Exp_14_Dec_2019_0845am-26.0.jpg  negatives


In [11]:
#Data_Dir
data_dir = '/media/DOLPHIN_ALEXIS/Analyses_alexis/Spectrograms_datasets/dataset/_last/tests/train/'

# Defining data generator with Data Augmentation
data_gen_augmented = ImageDataGenerator(rescale = 1/255., validation_split = 0.2)

print('Augmented training Images:')
train_ds = data_gen_augmented.flow_from_directory(data_dir, 
                                                              target_size = (224, 224), 
                                                              batch_size = 32,
                                                              subset = 'training',
                                                              class_mode = 'categorical')

#Testing Augmented Data
# Defining Validation_generator withour Data Augmentation
data_gen = ImageDataGenerator(rescale = 1/255., validation_split = 0.2)

print('Unchanged Validation Images:')
validation_ds = data_gen.flow_from_directory(data_dir, 
                                        target_size = (224, 224), 
                                        batch_size = 32,
                                        subset = 'validation',
                                        class_mode = 'categorical')

#Testing Augmented Data
test_dir_path = "/media/DOLPHIN_ALEXIS/Analyses_alexis/Spectrograms_datasets/dataset/_last/tests/test"
# Defining Validation_generator withour Data Augmentation
data_test_gen = ImageDataGenerator(rescale = 1/255.)

print('Test Validation Images:')
test_ds = data_gen.flow_from_directory(test_dir_path, 
                                        target_size = (224, 224), 
                                        batch_size = 32,
                                        subset = 'validation',
                                        class_mode = 'categorical')

Augmented training Images:
Found 26035 images belonging to 3 classes.
Unchanged Validation Images:
Found 6507 images belonging to 3 classes.
Test Validation Images:
Found 1021 images belonging to 3 classes.


In [4]:
model_paper = tf.keras.models.load_model("/kaggle/input/cnn_v1_xception/tensorflow2/initial_model/1/model_vgg.h5") #tf & keras ==2.15.0

#Early_Stopping
early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=5, 
    restore_best_weights=True,
)

#Fitting Model
history = model_paper.fit(train_ds,
                        epochs= 15,
                        steps_per_epoch = len(train_ds),
                        validation_data = validation_ds,
                        validation_steps = len(validation_ds),
                        callbacks = early_stopping)

'history = model_paper.fit(train_ds,\n                        epochs= 15,\n                        steps_per_epoch = len(train_ds),\n                        validation_data = validation_ds,\n                        validation_steps = len(validation_ds),\n                        callbacks = early_stopping)'